
# DL-Fall-25 — Clean Notebook (macOS-friendly)
**TinyLlama SFT (LoRA, no bitsandbytes) → submission.csv**

Runs locally on macOS (no CUDA). Trains a small open model (TinyLlama 1.1B) with PEFT LoRA and writes a Kaggle-ready `submission.csv`.


## 0) Environment check

In [1]:

import platform, torch, sys
print("Python :", platform.python_version())
print("OS     :", platform.platform())
print("CUDA   :", torch.version.cuda)
print("GPU    :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "NO CUDA GPU")
print("MPS    :", getattr(torch.backends, "mps", None) is not None and torch.backends.mps.is_available())
print("Executable:", sys.executable)


Python : 3.13.3
OS     : macOS-15.6.1-arm64-arm-64bit-Mach-O
CUDA   : None
GPU    : NO CUDA GPU
MPS    : True
Executable: /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python


## 1) Clean & install deps (no SciPy/sklearn; pinned transformers/TRl)

In [2]:

%pip -q uninstall -y scipy scikit-learn
%pip -q install -U "numpy>=1.26,<3.0" "pandas>=2.1.0" "datasets>=2.20.0" "accelerate>=0.33.0" "transformers==4.40.2" "trl==0.9.6" "peft>=0.12.0" "evaluate>=0.4.2" "torch>=2.3.0"
print("✅ Deps installed.")


Note: you may need to restart the kernel to use updated packages.
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [99 lines of output]
      Running `maturin pep517 build-wheel -i /opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/bin/python --compatibility off`
      Python reports SOABI: cpython-313-darwin
      Computed rustc target triple: aarch64-apple-darwin
      Installation directory: /Users/yuktakulkarni/Library/Caches/puccinialin
      Rustup already downloaded
      Installing rust to /Users/yuktakulkarni/Library/Caches/puccinialin/rustup
      warn: It looks like you have an existing rustup settings file at:
      warn: /Users/yuktakulkarni/.rustup/settings.toml
      warn: Rustup will install the default toolchain as specified in the settings file,
      warn: instead of the one inferred from the default host triple.
      info: profile set to 'minimal'
      info: default host tripl

## 2) Import libs (block optional sklearn path)

In [3]:
# --- Fix: disable TF/Flax in Transformers and purge TF/Keras from this kernel ---
%pip -q uninstall -y keras tf-keras tensorflow tensorflow-macos tensorflow-metal

import os, sys
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

# If parts of these libs were already imported, drop them so re-imports use new flags
for m in list(sys.modules):
    if m.startswith(("keras", "tensorflow", "transformers")):
        sys.modules.pop(m, None)

print("✅ Disabled TF/Flax and cleared related modules.")


Note: you may need to restart the kernel to use updated packages.
✅ Disabled TF/Flax and cleared related modules.


In [4]:
import transformers.utils.import_utils as iu
iu.is_sklearn_available = lambda: False  # keep sklearn out

from datasets import load_dataset, Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, LogitsProcessor
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, TaskType
import os, random, numpy as np, torch, pandas as pd
print("✅ Imports OK")


/opt/homebrew/Cellar/jupyterlab/4.3.4_1/libexec/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports OK


## 3) Load dataset (public HF)

In [5]:

ds = load_dataset("ad6398/nyu-dl-teach-maths-comp")
train_ds, test_ds = ds["train"], ds["test"]
print(train_ds); print(test_ds)


Dataset({
    features: ['question', 'is_correct', 'answer', 'solution'],
    num_rows: 1000000
})
Dataset({
    features: ['question', 'is_correct', 'answer', 'solution'],
    num_rows: 10000
})


## (Optional) Local CSV loader (uncomment and set paths)

In [6]:

# TRAIN_CSV = "train.csv"; TEST_CSV  = "test.csv"
# import pandas as pd
# df_train = pd.read_csv(TRAIN_CSV); df_test  = pd.read_csv(TEST_CSV)
# df_train["is_correct"] = (df_train["is_correct"].astype(str).str.strip().str.lower()
#                           .map({"true": True, "false": False, "1": True, "0": False}))
# from datasets import Dataset
# train_ds = Dataset.from_pandas(df_train, preserve_index=False)
# test_ds  = Dataset.from_pandas(df_test,  preserve_index=False)
# print(train_ds); print(test_ds)


## 4) Config

In [7]:

from dataclasses import dataclass
@dataclass
class Config:
    model_id: str = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    out_dir: str = "outputs"
    use_solution: int = 1
    epochs: int = 1
    lr: float = 2e-4
    per_device_bs: int = 1
    grad_accum: int = 8
    max_len: int = 1024
    seed: int = 42
    report_to: str = "none"
    TRAIN_LIMIT: int | None = 5000
    VAL_LIMIT:   int | None = 500
    TEST_LIMIT:  int | None = None
cfg = Config(); cfg


Config(model_id='TinyLlama/TinyLlama-1.1B-Chat-v1.0', out_dir='outputs', use_solution=1, epochs=1, lr=0.0002, per_device_bs=1, grad_accum=8, max_len=1024, seed=42, report_to='none', TRAIN_LIMIT=5000, VAL_LIMIT=500, TEST_LIMIT=None)

## 5) Utilities

In [8]:

def seed_everything(seed: int = 42):
    import random, numpy as np
    random.seed(seed); np.random.seed(seed)

from transformers import LogitsProcessor
import torch, numpy as np, random

SYS_PROMPT = (
    "You are a precise math answer verifier. "
    "Given a problem, a student's answer, and an optional worked solution, "
    "respond with a single digit label without explanation:\n"
    "- 1 if the student's answer is correct\n"
    "- 0 if the student's answer is incorrect"
)

def build_user_block(example, use_solution: bool = True) -> str:
    q = (example.get("question") or "").strip()
    a = (example.get("answer") or "").strip()
    sol = example.get("solution") or ""
    parts = [f"Problem:\n{q}\n\nStudent Answer:\n{a}"]
    if use_solution and isinstance(sol, str) and sol.strip():
        parts.append(f"\nSolution (reference reasoning):\n{sol.strip()}")
    parts.append("\nReturn only the digit (1 or 0).")
    return "\n".join(parts)

def make_text(example, use_solution: bool, label=None):
    hdr = f"<<SYS>>\n{SYS_PROMPT}\n<</SYS>>\n\n"
    usr = f"<<USR>>\n{build_user_block(example, use_solution)}\n<</USR>>\n\n"
    asst = "<<ASSISTANT>>\n" + (str(int(label)) if label is not None else "")
    return hdr + usr + asst

class AllowedTokensLogitsProcessor(LogitsProcessor):
    def __init__(self, allowed_token_ids): self.allowed = set(allowed_token_ids)
    def __call__(self, input_ids, scores):
        mask = torch.full_like(scores, float("-inf"))
        for tid in self.allowed:
            if tid < scores.shape[-1]:
                mask[..., tid] = 0.0
        return scores + mask

def get_allowed_token_ids(tokenizer):
    return [tokenizer.convert_tokens_to_ids(t) for t in ["0", "1"]]

seed_everything(42)
print("✅ Utils ready")


✅ Utils ready


## 6) Split (ClassLabel + subsets)

In [9]:

train_ds = train_ds.class_encode_column("is_correct")
dd = train_ds.train_test_split(test_size=0.05, seed=42, stratify_by_column="is_correct")
if cfg.TRAIN_LIMIT:
    dd["train"] = dd["train"].shuffle(seed=42).select(range(cfg.TRAIN_LIMIT))
if cfg.VAL_LIMIT:
    dd["test"]  = dd["test"].shuffle(seed=42).select(range(cfg.VAL_LIMIT))
dataset = DatasetDict({"train": dd["train"], "validation": dd["test"]})
dataset


DatasetDict({
    train: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['question', 'is_correct', 'answer', 'solution'],
        num_rows: 500
    })
})

## 7) Tokenizer + model (MPS if available) + LoRA + tokenize

In [10]:

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(cfg.model_id, use_fast=True)  # <- fast tokenizer, no sentencepiece needed
tokenizer.padding_side = "right"
tokenizer.truncation_side = "left"


import torch
device_map = "mps" if getattr(torch.backends, "mps", None) and torch.backends.mps.is_available() else "auto"
model = AutoModelForCausalLM.from_pretrained(cfg.model_id, device_map=device_map)

from peft import LoraConfig, get_peft_model, TaskType
peft_cfg = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05, bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
)
model = get_peft_model(model, peft_cfg)
model.print_trainable_parameters()

def preprocess(example, use_solution: bool):
    label = 1 if example["is_correct"] else 0
    return {"text": make_text(example, use_solution=bool(use_solution), label=label)}

proc_train = dataset["train"].map(lambda x: preprocess(x, True))
proc_val   = dataset["validation"].map(lambda x: preprocess(x, True))
len(proc_train), len(proc_val)


trainable params: 12,615,680 || all params: 1,112,664,064 || trainable%: 1.1338


(5000, 500)

## 8) Train (TRL SFT) + save adapter

In [11]:
# ==== FAST TRAIN (subset + few steps) ====
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer

# use small subset
N_TRAIN = 600     # reduce to taste
N_VAL   = 0       # skip val to save time (set 100 if you want a tiny val)
MAX_LEN = 384     # shorter sequences are faster
MAX_STEPS = 120   # hard cap; overrides epochs if > 0

# build subset from your *preprocessed* data
small_train = proc_train.select(range(min(N_TRAIN, len(proc_train))))
small_val   = proc_val.select(range(min(N_VAL,   len(proc_val))))

def tok_fast(batch):
    out = tokenizer(batch["text"], truncation=True, max_length=MAX_LEN)
    out["labels"] = out["input_ids"].copy()
    return out

tok_train_fast = small_train.map(tok_fast, batched=True, remove_columns=small_train.column_names)
tok_val_fast   = small_val.map(tok_fast,   batched=True, remove_columns=small_val.column_names) if N_VAL>0 else None

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

train_args_fast = TrainingArguments(
    output_dir="outputs_fast",
    num_train_epochs=0.5,              # ignored if MAX_STEPS>0
    max_steps=MAX_STEPS,               # keep small
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=cfg.lr,
    logging_steps=50,
    save_steps=10_000_000,             # effectively disable saves during train
    save_total_limit=1,
    fp16=False, bf16=False,
    dataloader_pin_memory=False,       # silence MPS warning
    report_to=[],                      # no logging backends
)

trainer_fast = Trainer(
    model=model,                       # your LoRA-wrapped model
    args=train_args_fast,
    train_dataset=tok_train_fast,
    eval_dataset=tok_val_fast,         # None if N_VAL=0
    data_collator=data_collator,
)

trainer_fast.train()

# save adapter separately
import os
adapter_fast_dir = os.path.join("outputs_fast", "adapter")
model.save_pretrained(adapter_fast_dir)
tokenizer.save_pretrained("outputs_fast")
print(f"✅ Fast training done. Saved LoRA adapter to: {adapter_fast_dir}")


The model is already on multiple devices. Skipping the move to device specified in `args`.


Step,Training Loss
50,0.889500
100,0.788600


✅ Fast training done. Saved LoRA adapter to: outputs_fast/adapter


## 9) Inference → `submission.csv`

In [16]:
# ===== Context-aware no-generate inference (CPU, left-pad) =====
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch, os, pandas as pd

# knobs
TEST_LIMIT    = 1000      # set to e.g. 1000 for quick test; None = full 10k
BATCH_SIZE    = 4         # safe; bump to 8 if smooth
MAX_LEN_INFER = 192

# tokenizer settings for decoder-only models
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
    tokenizer.pad_token = tokenizer.eos_token

DEVICE = torch.device("cpu")
torch.set_num_threads(max(1, (os.cpu_count() or 4) - 1))

# reload base + LoRA on CPU
base = AutoModelForCausalLM.from_pretrained(cfg.model_id, dtype=torch.float32, low_cpu_mem_usage=True)
model_inf = PeftModel.from_pretrained(base, os.path.join("outputs_fast", "adapter"))
model_inf.to(DEVICE).eval()

def next_token_id_in_context(prompt: str, digit: str) -> int:
    """
    Get the FIRST token id the tokenizer would emit for continuing `prompt` with `digit`
    under the SAME truncation/left-padding regime used in inference.
    """
    ids_base = tokenizer(prompt, add_special_tokens=False, truncation=True,
                         max_length=MAX_LEN_INFER)["input_ids"]
    ids_plus = tokenizer(prompt + digit, add_special_tokens=False, truncation=True,
                         max_length=MAX_LEN_INFER)["input_ids"]
    # With left truncation, we can't rely on lengths lining up; take last id as the continuation token.
    if not ids_plus:
        return None
    return ids_plus[-1]

# choose test slice
ds = test_ds if TEST_LIMIT is None else test_ds.select(range(min(TEST_LIMIT, len(test_ds))))

preds = []
N = len(ds)
for start in range(0, N, BATCH_SIZE):
    end = min(start + BATCH_SIZE, N)
    prompts = [make_text(ds[i], use_solution=bool(cfg.use_solution), label=None) for i in range(start, end)]

    # context-aware ids for each row
    id0_list = [next_token_id_in_context(p, "0") for p in prompts]
    id1_list = [next_token_id_in_context(p, "1") for p in prompts]

    # encode prompts
    enc = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_LEN_INFER)
    enc = {k: v.to(DEVICE) for k, v in enc.items()}

    with torch.inference_mode():
        out = model_inf(**enc)             # [B, L, V]
        logits = out.logits
        last_idx = enc["attention_mask"].sum(dim=1) - 1
        B = logits.size(0)
        last = logits[torch.arange(B, device=DEVICE), last_idx]   # [B, V]

        batch_preds = []
        for b in range(B):
            t0 = id0_list[b]; t1 = id1_list[b]
            # if any id is None, fall back to old simple ids
            if t0 is None or t1 is None:
                # robust fallback: try raw digit id, space+digit, newline+digit
                cand = []
                for s in ("0", " 0", "\n0"):
                    tok = tokenizer.encode(s, add_special_tokens=False)
                    if tok: cand.append(tok[-1])
                t0 = (cand[-1] if cand else None)

                cand = []
                for s in ("1", " 1", "\n1"):
                    tok = tokenizer.encode(s, add_special_tokens=False)
                    if tok: cand.append(tok[-1])
                t1 = (cand[-1] if cand else None)

            # if still missing, default to False
            if t0 is None or t1 is None or t0 >= last.shape[-1] or t1 >= last.shape[-1]:
                batch_preds.append(False)
                continue

            batch_preds.append(last[b, t1].item() > last[b, t0].item())

    preds.extend({"ID": i, "is_correct": bool(p)} for i, p in zip(range(start, end), batch_preds))
    if (start // BATCH_SIZE) % 100 == 0:
        print(f"{end}/{N}")

# save CSV
os.makedirs("outputs_fast", exist_ok=True)
sub_path = os.path.join("outputs_fast", "submission.csv")
pd.DataFrame(preds, columns=["ID","is_correct"]).to_csv(sub_path, index=False)
print(f"Saved submission to: {sub_path}  (rows={len(preds)})")


4/1000
404/1000
804/1000
Saved submission to: outputs_fast/submission.csv  (rows=1000)


## 10) Validate submission

In [17]:
import pandas as pd, os
p = os.path.join("outputs_fast", "submission.csv")
df = pd.read_csv(p)
display(df.head())
print("Rows:", len(df), "Unique IDs:", df["ID"].nunique())
assert set(df["is_correct"].unique()).issubset({True, False})
print("✅ Submission looks good at", p)


,ID,is_correct
0,0,False
1,1,False
2,2,False
3,3,False
4,4,False


Rows: 1000 Unique IDs: 1000
✅ Submission looks good at outputs_fast/submission.csv



---
## Kaggle GPU quick swap (Llama-3-8B + QLoRA)
1) Settings: GPU **T4**, Internet **On**; add Secret **HF_TOKEN**.  
2) Install:
```
%pip -q install -U "accelerate>=0.33.0" "bitsandbytes>=0.43.0" "datasets>=2.20.0" "evaluate>=0.4.2" "numpy>=1.26.0" "pandas>=2.1.0" "peft>=0.12.0" "scikit-learn>=1.3.0" "transformers>=4.44.0" "trl>=0.10.1"
```
3) Set:
```
cfg.model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
cfg.per_device_bs, cfg.grad_accum, cfg.max_len = 1, 16, 2048
from huggingface_hub import login; import os; login(os.environ["HF_TOKEN"])
```
4) Re-run from **dataset** onward.


In [14]:
# Robust SFTConfig + SFTTrainer setup across TRL versions
from inspect import signature
from trl import SFTTrainer, SFTConfig
import os

# ---------- 1) Build SFTConfig with only stable keys, and add eval/log if supported ----------
cfg_kwargs = dict(
    output_dir=cfg.out_dir,
    num_train_epochs=cfg.epochs,
    learning_rate=cfg.lr,
    per_device_train_batch_size=cfg.per_device_bs,
    per_device_eval_batch_size=max(1, cfg.per_device_bs//2),
    gradient_accumulation_steps=cfg.grad_accum,
    logging_steps=25,
    save_steps=200,
    save_total_limit=2,
    bf16=False, fp16=False, tf32=False,     # safe on CPU/MPS
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    gradient_checkpointing=True,
    report_to=cfg.report_to,
)

sig_cfg = signature(SFTConfig)
# evaluation arg changed across versions; add the one your TRL exposes
if "evaluation_strategy" in sig_cfg.parameters:
    cfg_kwargs["evaluation_strategy"] = "steps"
elif "eval_strategy" in sig_cfg.parameters:
    cfg_kwargs["eval_strategy"] = "steps"
# Some TRL versions accept max_length here; others want it on the trainer. We'll prefer trainer side.
if "max_length" in sig_cfg.parameters:
    cfg_kwargs["max_length"] = cfg.max_len
# (avoid max_seq_length here; we’ll set it on the trainer where it’s more stable)

sft_config = SFTConfig(**cfg_kwargs)

# ---------- 2) Build SFTTrainer with dataset-specific args passed here ----------
trainer_kwargs = dict(
    model=model,
    tokenizer=tokenizer,
    args=sft_config,
    train_dataset=proc_train,
    eval_dataset=proc_val,   # harmless if eval strategy is 'no'
)

sig_trainer = signature(SFTTrainer.__init__)
# Pass dataset_text_field/packing if this TRL exposes them on the trainer
if "dataset_text_field" in sig_trainer.parameters:
    trainer_kwargs["dataset_text_field"] = "text"
if "packing" in sig_trainer.parameters:
    trainer_kwargs["packing"] = True

# max length key varies: some versions use max_seq_length, others max_length on trainer
if "max_seq_length" in sig_trainer.parameters:
    trainer_kwargs["max_seq_length"] = cfg.max_len
elif "max_length" in sig_trainer.parameters:
    trainer_kwargs["max_length"] = cfg.max_len

trainer = SFTTrainer(**trainer_kwargs)
trainer.train()

adapter_dir = os.path.join(cfg.out_dir, "adapter")
trainer.model.save_pretrained(adapter_dir)
tokenizer.save_pretrained(cfg.out_dir)
print(f"✅ Trained. Saved LoRA adapter to: {adapter_dir}")


TypeError: SFTTrainer.__init__() got an unexpected keyword argument 'tokenizer'

In [ ]:

from peft import PeftModel
base = AutoModelForCausalLM.from_pretrained(cfg.model_id, device_map=device_map)
model_inf = PeftModel.from_pretrained(base, os.path.join(cfg.out_dir, "adapter")); model_inf.eval()

allowed_ids = get_allowed_token_ids(tokenizer)
logits_proc = AllowedTokensLogitsProcessor(allowed_ids)
gen_cfg = GenerationConfig(max_new_tokens=1, do_sample=False, temperature=0.0, top_p=1.0,
                           eos_token_id=[tokenizer.eos_token_id], pad_token_id=tokenizer.eos_token_id)

test_for_pred = test_ds if cfg.TEST_LIMIT is None else test_ds.select(range(cfg.TEST_LIMIT))
preds = []
for i, ex in enumerate(test_for_pred):
    prompt = make_text(ex, use_solution=True, label=None)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=cfg.max_len)
    inputs = {k: v.to(model_inf.device) for k, v in inputs.items()}
    with torch.no_grad():
        out = model_inf.generate(**inputs, generation_config=gen_cfg, logits_processor=[logits_proc])
    new_tokens = out[0, inputs["input_ids"].shape[1]:]
    text = tokenizer.decode(new_tokens, skip_special_tokens=True)
    ch = (text.strip()[:1] if text.strip() else "0")
    preds.append({"ID": i, "is_correct": (ch == "1")})

import pandas as pd, os
os.makedirs(cfg.out_dir, exist_ok=True)
sub_path = os.path.join(cfg.out_dir, "submission.csv")
pd.DataFrame(preds, columns=["ID","is_correct"]).to_csv(sub_path, index=False)
print(f"Saved submission to: {sub_path}")


In [18]:
import os, sys
print("Notebook working dir:", os.getcwd())
!ls -la


Notebook working dir: /Users/yuktakulkarni/DL_Midterm
The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.
total 1056
drwxr-xr-x@   9 yuktakulkarni  staff     288 Oct 26 19:37 .
drwxr-x---+ 132 yuktakulkarni  staff    4224 Oct 26 19:37 ..
drwxr-xr-x@   6 yuktakulkarni  staff     192 Oct 26 19:37 .ipynb_checkpoints
-rw-r--r--@   1 yuktakulkarni  staff   46338 Oct 26 12:06 DL_Fall25_Clean_mac_TinyLlama_SFT_to_Submission.ipynb
-rw-r--r--@   1 yuktakulkarni  staff  107081 Oct 26 00:07 DL_Fall25_Llama3_SFT_QLoRA_EndToEnd_v3_mac_or_kaggle.ipynb
drwxr-xr-x@   2 yuktakulkarni  staff      64 Oct 25 15:37 outputs
drwxr-xr-x@  12 yuktakulkarni  staff     384 Oct 26 11:41 outputs_fast
-rw-r--r--@   1 yuktakulkarni  staff  333904 Oct 25 15:04 sample_sub.csv
-rw-r--r--@   1 yuktakulkarni  staff   44690 Oct 26 00:12 Untitled.ipynb


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [19]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
try:
    get_ipython().run_line_magic("config", "HistoryManager.enabled = False")
except Exception:
    pass
print("✅ Ready")


✅ Ready
